In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import VGG19, ResNet50, MobileNetV2 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, ReduceLROnPlateau

In [2]:
# Load data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
val = pd.read_csv('data/val.csv')

In [3]:
# Define emotion dictionary and labels
emotion_dict = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
emotion_labels = list(emotion_dict.values())

# Convert pixel values to arrays and normalize
def preprocess_data(df):
    df['pixels'] = df['pixels'].apply(lambda x: np.array(x.split(), dtype='float32'))
    df['pixels'] = df['pixels'] / 255.0
    return np.array(df['pixels'].tolist()).reshape(-1, 48, 48, 1)

In [4]:
X_train = preprocess_data(train)
X_test = preprocess_data(test)
X_val  = preprocess_data(val)

y_train = to_categorical(train['emotion'], num_classes=7)
y_test = to_categorical(test['emotion'], num_classes=7)
y_val = to_categorical(val['emotion'], num_classes=7)

In [5]:
# Resize images to 224x224
def resize_images(X, size=(224, 224)):
    return tf.image.resize(X, size).numpy()

## Example resizing and conversion to RGB
#X_train_resized = np.stack([np.tile(np.expand_dims(tf.image.resize(x, (224, 224)), axis=-1), (1, 1, 3)) for x in X])


X_train_resized = resize_images(X_train)
X_test_resized = resize_images(X_test)
X_val_resized = resize_images(X_val)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)



In [ ]:
# Define models
def vgg19_model():
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) #input_shape=(224, 224, 3)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(7, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def resnet50_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) #input_shape=(224, 224, 3)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(7, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def mobilenetv2_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) #input_shape=(224, 224, 3)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(7, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def cnn_model():
    model = Sequential([
        Input(shape=(48, 48, 1)),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Callbacks
def on_epoch_end(epoch, logs):
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}:")
        print(f" - loss: {logs['loss']:.4f} - accuracy: {logs['accuracy']:.4f}")
        print(f" - val_loss: {logs['val_loss']:.4f} - val_accuracy: {logs['val_accuracy']:.4f}")

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=6, min_lr=0.0001)

In [ ]:
# Train and evaluate models
models = {
    'CustomCNN': cnn_model(),
    'VGG19': vgg19_model(),
    'ResNet50': resnet50_model(),
    'MobileNetv2': mobilenetv2_model()
}

histories = {}

for name, model in models.items():
    print(f"Training {name}...")
    if name in ['VGG19', 'ResNet50','MobileNetv2']:
        history = model.fit(
            datagen.flow(X_train_resized, y_train, batch_size=64), #X_train_resized
            epochs=20,
            validation_data=(X_val_resized, y_val), #X_val_resized
            callbacks=[print_callback, early_stopping, reduce_lr]
        )
    else:
        history = model.fit(
            datagen.flow(X_train, y_train, batch_size=64),
            epochs=20,
            validation_data=(X_val, y_val),
            callbacks=[print_callback, early_stopping, reduce_lr]
        )
    histories[name] = history

In [ ]:
# Evaluate models
for name, model in models.items():
    print(f"Evaluating {name}...")
    if name in ['VGG19', 'ResNet50','MobileNetv2']:
        loss, accuracy = model.evaluate(X_test_resized, y_test) #X_test_resized
        y_pred = model.predict(X_test)
    else:
        loss, accuracy = model.evaluate(X_test, y_test)
        y_pred = model.predict(X_test)
    
    print(f"Accuracy of the {name} model is - {accuracy * 100:.2f}%")

    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
                xticklabels=emotion_labels, yticklabels=emotion_labels) 
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix for {name}')
    plt.show()

    # Save models
    model.save(f'{name}_model.h5')
